In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
import pickle
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import import_ipynb
import wiktionary_parser
from statistics import mean
import preprocess_utils
import wikipedia
import pandas as pd
import numpy as np

from gensim.models import Phrases
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

importing Jupyter notebook from wiktionary_parser.ipynb
importing Jupyter notebook from preprocess_utils.ipynb


In [ ]:
wiktionary_parser('hi')

In [ ]:
# accuracy scores:
# title to domain name (word2vec) = 75%
# text to def sentence (doc2vec) = 6.25%
# text to domain + sentence (doc2vec) = 6.25%

In [54]:
def model_title_domain_gen(title, domain, pos):
    definition_sents = wiktionary_parser.define(domain)[pos]
    raw_domains = [nltk.word_tokenize(sent[:sent.find(')')].replace('(','').replace(')','').lower()) for sent in definition_sents]
    domains = []
    for arr in raw_domains:
        temp = []
        for word in arr:
            if word not in stop_words and word!=',':
                temp.append(word)
        domains.append(temp)

    title_tokens = nltk.word_tokenize(title.lower())


    FINAL = []
    for dt in domains:
        temp2 = []
        for dtt in dt:
            temp1 = []
            for tt in title_tokens:
                try:
                    similarity = glove_vectors.similarity(dtt,tt)
                except:
                    similarity = 0
                        
                temp1.append(similarity)
            temp2.append(np.mean(temp1))
        FINAL.append(np.mean(temp2))
    return FINAL, definition_sents


In [3]:
def model_gen(curtext):
    sents = []
    for sent in nltk.sent_tokenize(curtext):
        sent = sent[:-1].lower()
        temp = []
        for word in nltk.word_tokenize(sent):
            if word not in stop_words:
                temp.append(lemmatizer.lemmatize(word))
        sents.append(temp)

    tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(sents)]

    model = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs = 100)
    
    return model

In [49]:
def model_text_domain_plus_sent_predict(model, kw, pos):
    term_to_define = wikt_def_gen.extract(kw)[pos]
    
    final = []
    
    for candidate in term_to_define:
        def_sent_candidate = []
        candidate = candidate[:-1].lower()
        for word in word_tokenize(preprocess_utils.clean_text_keep_bracket_content(candidate)):
            if word not in stop_words:
                def_sent_candidate.append(lemmatizer.lemmatize(word))
    
        if def_sent_candidate[0] == 'general':
            del def_sent_candidate[0]
        print([' '.join(def_sent_candidate), mean([obj[1] for obj in model.docvecs.most_similar(positive=[model.infer_vector(def_sent_candidate)], topn = 10000)])])
#         print(result)
#         final.append(result[1])
        
        final.append(mean([obj[1] for obj in model.docvecs.most_similar(positive=[model.infer_vector(def_sent_candidate)], topn = 10000)]))
        
    return final, term_to_define

In [45]:
def model_text_sent_nodomain_predict(model, kw, pos):
    term_to_define = wikt_def_gen.extract(kw)[pos]
    
    final = []
    
    for candidate in term_to_define:
        def_sent_candidate = []
        candidate = candidate[:-1].lower()
        for word in word_tokenize(preprocess_utils.clean_text(candidate)):
            if word not in stop_words:
                def_sent_candidate.append(lemmatizer.lemmatize(word))
        
        print([' '.join(def_sent_candidate), mean([obj[1] for obj in model.docvecs.most_similar(positive=[model.infer_vector(def_sent_candidate)], topn = 10000)])])
#         print(result)
#         final.append(result[1])
        
        final.append(mean([obj[1] for obj in model.docvecs.most_similar(positive=[model.infer_vector(def_sent_candidate)], topn = 10000)]))
        
    return final, term_to_define

In [43]:
# temp_text = preprocess_utils.clean_text(wikipedia.page('Computers').content.lower())
# model = model_gen(temp_text)
# model_text_domain_plus_sent_predict(model, 'mouse', 'noun')

In [8]:
with open('/home/jackragless/projects/data/DAIC_GLOGEN/doc2vec_eval_data.pkl', 'rb') as f:
    eval_data = pickle.load(f)

In [50]:
score = 0
count = 0
for subject in eval_data.keys():
    temp_text = preprocess_utils.clean_text(wikipedia.page(subject).content)
    model = model_gen(temp_text)
    for i in eval_data[subject]:
        count += 1
        temp_prob_arr, def_sents = model_text_domain_plus_sent_predict(model, i[0], i[1])
        temp_index = np.where(temp_prob_arr == np.max(temp_prob_arr))[0][0]
        guess = def_sents[temp_index]
#         print(i[0], '<--->', i[2], '<--->', guess)
        if guess.strip() == i[2].strip():
            score+=1
print(100*(score/count))

['thing physical existence', 0.9664289238526649]
['objective goal end purpose something', 0.9699825529790006]
['grammar noun phrase internal complement verb phrase prepositional phrase . verb phrase transitive action verb typically receiver action', 0.9722101059566248]
['person thing toward emotion directed', 0.9450747381985393]
['object-oriented programming instantiation class structure', 0.8362986482295793]
['category theory element within category upon function operate . thus category consists set element object function operate', 0.9507089945458874]
['obsolete sight show appearance aspect', 0.9719713334365399]
['countable group collection category set sharing characteristic attribute', 0.9420202745195106]
['sociology countable social grouping based job wealth etc . britain society commonly split three main class upper class middle class working class', 0.9456142686896308]
['uncountable division society class', 0.8670814771627643]
['uncountable admirable behavior elegance', -0.19027

['small rodent genus mu', 0.8824144021137473]
['informal member many small rodent marsupial specie resembling rodent', 0.7857928474232093]
['quiet shy person', 0.8810056417258745]
['computing plural mouse rarely mouse input device moved pad flat surface produce corresponding movement pointer graphical display', 0.652657186545278]
['boxing facial hematoma black eye', 0.2822106143978146]
['nautical turn lashing spun yarn small stuff metallic clasp fastening uniting point shank hook prevent unhooking straightening', 0.8497004378779033]
['obsolete familiar term endearment', 0.82993598144079]
['match used firing gun blasting', 0.8484994511782509]
['set theory small model fragment zermelo-fraenkel set theory desirable property depending context', 0.6883599430169027]
['historical small cushion woman hair', 0.8617361909949902]
['assign duty responsibility', 0.894186793298451]
['transitive assign debit accoun', 0.15741873826572345]
['transitive pay account using credit car', 0.7577389812039346]

In [55]:
score = 0
count = 0
for subject in eval_data.keys():
    for i in eval_data[subject]:
        count += 1
        temp_prob_arr, def_sents = model_title_domain_gen(subject, i[0], i[1])
        temp_index = np.where(temp_prob_arr == np.max(temp_prob_arr))[0][0]
        guess = def_sents[temp_index]
        print(i[0], '<--->', i[2], '<--->', guess)
        if guess.strip() == i[2].strip():
            print(1,'\n')
            score+=1
        else:
            print(0,'\n')
print(100*(score/count))

object <---> (object-oriented programming) An instantiation of a class or structure. <---> (object-oriented programming) An instantiation of a class or structure.
1 

class <---> (object-oriented programming, countable) A set of objects having the same behavior (but typically differing in state), or a template defining such a set. <---> (object-oriented programming, countable) A set of objects having the same behavior (but typically differing in state), or a template defining such a set.
1 

paradigm <---> An example serving as the model for such a pattern.  <---> (linguistics) A set of all forms which contain a common element, especially the set of all inflectional forms of a word or a particular grammatical category.
0 

loop <---> (programming) A programmed sequence of instructions that is repeated until or while a particular condition is satisfied. <---> (programming) A programmed sequence of instructions that is repeated until or while a particular condition is satisfied.
1 

shor

In [ ]:
# wikt_def_gen.extract('forte')

In [ ]:
# curtext = preprocess_utils.clean_text(wikipedia.page('Australia').content)
# curtext

In [ ]:
#compare doc to definition domain
# for candidate in term_to_define:
#     final_candidate = []
#     final_candidate = candidate[candidate.find('(')+1:candidate.find(')')].replace(',','').lower().split()
#     print(' '.join(final_candidate), '<--->', mean([obj[1] for obj in model.docvecs.most_similar(positive=[model.infer_vector(final_candidate)], topn = len(tagged_data))]))

In [ ]:
#compare doc to definition
# for candidate in term_to_define:
#     final_candidate = []
#     final_candidate = candidate[candidate.find(')')+1:].replace(',','').lower().split()
#     print(' '.join(final_candidate), '<--->', mean([obj[1] for obj in model.docvecs.most_similar(positive=[model.infer_vector(final_candidate)], topn = len(tagged_data))]))

In [ ]:
# def model_text_domain_predict(model, kw, pos):
#     term_to_define = wikt_def_gen.extract(kw)[pos]
    
#     final = []
    
#     for candidate in term_to_define:
#         def_sent_candidate = []
#         candidate = candidate[:-1].lower()
#         candidate = candidate[candidate.find('(')+1:candidate.find(')')].replace(',','').lower().split()
        
#         final.append([candidate, mean([obj[1] for obj in model.docvecs.most_similar(positive=[model.infer_vector(candidate)], topn = 10000)])])
        
#     return final